In [1]:
import numpy as np
import scipy
import scipy.spatial
import scipy.ndimage
import nibabel as nib
import random
import pickle
import os
import sklearn
from sklearn import decomposition
import joblib

In [2]:
# gm mask
gm_mask = nib.load('all_vols_mask.nii')
gm_mask_data = gm_mask.get_fdata()
gm_vox = np.nonzero(gm_mask_data)
gm_vox_inds = np.ravel_multi_index(gm_vox, (91,109,91))

In [3]:
# load in brainnetome/SUIT ROIs
input_rois = nib.load('all_vols.nii')
input_rois_d = input_rois.get_fdata()
roi_inds = []
for i in range(273):
    roi_inds.append(np.nonzero(input_rois_d==i+1))

In [17]:
# NOTE: the preprocessed BOLD images used in this study aren't including in the github repository because they are very big
# contact me at jesse.brown@ucsf.edu if you really need them 
#bold_filelist = np.loadtxt('hcp/3d_vols_task/wm_filelist_200_smoothed.txt', dtype='str')
#vol_out_dir = 'hcp/3d_vols_task/wm/'
#bold_filelist = np.loadtxt('hcp/3d_vols_task/motor_filelist_200_smoothed.txt', dtype='str')
#vol_out_dir = 'hcp/3d_vols_task/motor/'
#bold_filelist = np.loadtxt('hcp/3d_vols_task/language_filelist_200_smoothed.txt', dtype='str')
#vol_out_dir = 'hcp/3d_vols_task/language/'
bold_filelist = np.loadtxt('hcp/3d_vols_task/emotion_filelist_200_smoothed.txt', dtype='str')
vol_out_dir = 'hcp/3d_vols_task/emotion/'

In [19]:
# read in 4d volumes
# standardize voxels in 4d to mean=0, std=1
# mask by gray matter, set nan to 0
# save as individual 3d volume files
# store 273 ROI mean timeseries in single across-subject array
n_vols = 171#311#279#400
n_regions = 273
roi_img_means = np.zeros((200*n_vols, n_regions))
mni_file = nib.load('MNI152_T1_2mm.nii')
mni_file_header = mni_file.header.copy()
for i in range(200):
    start_num = (n_vols*i) + 1
    bold_file = bold_filelist[i]
    bold_file = nib.load(bold_file)
    bold_data = bold_file.get_fdata()
    bold_data_norm = (bold_data - (np.mean(bold_data,3)[:,:,:,np.newaxis]))
    bold_data_norm = (bold_data_norm / (np.std(bold_data_norm,3)[:,:,:,np.newaxis]))
    bold_data_norm = bold_data_norm * gm_mask_data[:,:,:,np.newaxis]
    bold_data_norm[np.isnan(bold_data_norm)] = 0

    for j in range(n_vols):
        cur_img = bold_data_norm[:,:,:,j]
        bold_data_norm_nii = nib.Nifti1Image(cur_img, None, mni_file_header)
        out_filename = vol_out_dir + 'vol_%06d.nii.gz' %(start_num + j)
        nib.save(bold_data_norm_nii, out_filename)
        # get region mean timeseries
        for k in range(n_regions):
            roi_img_means[start_num + j - 1, k] = np.mean(cur_img[roi_inds[k][0],roi_inds[k][1],roi_inds[k][2]])
    print(i+1)
#np.save('roi_img_means_200_subjects_REST1.npy', roi_img_means)

<ipython-input-19-26b27108d2b0>:18: RuntimeWarning: invalid value encountered in true_divide
  bold_data_norm = (bold_data_norm / (np.std(bold_data_norm,3)[:,:,:,np.newaxis]))


1


<ipython-input-19-26b27108d2b0>:18: RuntimeWarning: invalid value encountered in true_divide
  bold_data_norm = (bold_data_norm / (np.std(bold_data_norm,3)[:,:,:,np.newaxis]))


2


<ipython-input-19-26b27108d2b0>:18: RuntimeWarning: invalid value encountered in true_divide
  bold_data_norm = (bold_data_norm / (np.std(bold_data_norm,3)[:,:,:,np.newaxis]))


...

198


<ipython-input-19-26b27108d2b0>:18: RuntimeWarning: invalid value encountered in true_divide
  bold_data_norm = (bold_data_norm / (np.std(bold_data_norm,3)[:,:,:,np.newaxis]))


199


<ipython-input-19-26b27108d2b0>:18: RuntimeWarning: invalid value encountered in true_divide
  bold_data_norm = (bold_data_norm / (np.std(bold_data_norm,3)[:,:,:,np.newaxis]))


200


In [20]:
# save ROI BOLD timeseries
#np.save('roi_img_means_200_subjects_REST1.npy', roi_img_means)
#np.save('roi_img_means_200_subjects_wm.npy', roi_img_means)
#np.save('roi_img_means_200_subjects_motor.npy', roi_img_means)
#np.save('roi_img_means_200_subjects_language.npy', roi_img_means)
#np.save('roi_img_means_200_subjects_emotion.npy', roi_img_means)

In [4]:
# load all task-free 3D BOLD images
train_set = range(119500) # discovery dataset
img_array = np.zeros((len(gm_vox_inds),len(train_set)))
for i in range(len(train_set)):
    img_filename = '3d_vols/vol_%06d.nii.gz' %(train_set[i]+1) # discovery dataset
    img_filename = '3d_vols/vol_%06d.nii.gz' %(train_set[i]+1+119500) # validation dataset
    img = nib.load(img_filename)
    img_d = img.get_fdata()
    img_array[:,i] = img_d[gm_vox[0], gm_vox[1], gm_vox[2]]
    if not np.mod(i,1000):
        print(i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000


In [21]:
# load all working memory task BOLD images
n_vols = 80000
img_array_wm = np.zeros((len(gm_vox_inds),n_vols)).astype('float32')
for i in range(n_vols):
    img_filename = 'alee/projects/hcp/3d_vols_task/wm/vol_%06d.nii.gz' %(i+1)
    img = nib.load(img_filename)
    img_d = img.get_fdata()
    img_array_wm[:,i] = img_d[gm_vox[0], gm_vox[1], gm_vox[2]]
    if not np.mod(i,5000):
        print(i)

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000


In [22]:
# load all motor task BOLD images
n_vols = 55800
img_array_motor = np.zeros((len(gm_vox_inds),n_vols)).astype('float32')
for i in range(n_vols):
    img_filename = 'alee/projects/hcp/3d_vols_task/motor/vol_%06d.nii.gz' %(i+1)
    img = nib.load(img_filename)
    img_d = img.get_fdata()
    img_array_motor[:,i] = img_d[gm_vox[0], gm_vox[1], gm_vox[2]]
    if not np.mod(i,5000):
        print(i)

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000


In [23]:
# load all language task BOLD images
n_vols = 62200
img_array_language = np.zeros((len(gm_vox_inds),n_vols)).astype('float32')
for i in range(n_vols):
    img_filename = '3d_vols_task/language/vol_%06d.nii.gz' %(i+1)
    img = nib.load(img_filename)
    img_d = img.get_fdata()
    img_array_language[:,i] = img_d[gm_vox[0], gm_vox[1], gm_vox[2]]
    if not np.mod(i,5000):
        print(i)

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000


In [24]:
# load all emotion task BOLD images
n_vols = 34200
img_array_emotion = np.zeros((len(gm_vox_inds),n_vols)).astype('float32')
for i in range(n_vols):
    img_filename = 'hcp/3d_vols_task/emotion/vol_%06d.nii.gz' %(i+1)
    img = nib.load(img_filename)
    img_d = img.get_fdata()
    img_array_emotion[:,i] = img_d[gm_vox[0], gm_vox[1], gm_vox[2]]
    if not np.mod(i,5000):
        print(i)

0
5000
10000
15000
20000
25000
30000


In [25]:
# stack all task BOLD images
img_array_comb_disc = np.vstack((img_array_wm[:,:40000].T,img_array_motor[:,:27900].T))
img_array_comb_disc = np.vstack((img_array_comb_disc,img_array_language[:,:31100].T))
img_array_comb_disc = np.vstack((img_array_comb_disc,img_array_emotion[:,:17100].T))

In [26]:
# check size of all defined python variables
import sys
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name, value in locals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

           img_array_comb_disc: 70.5 GiB
                  img_array_wm: 48.6 GiB
            img_array_language: 37.8 GiB
               img_array_motor: 33.9 GiB
             img_array_emotion: 20.8 GiB
                     bold_data:  1.1 GiB
                bold_data_norm:  1.1 GiB
                 roi_img_means: 71.2 MiB
                  gm_mask_data:  6.9 MiB
                  input_rois_d:  6.9 MiB


In [29]:
# pca on task-free voxel timeseries (the primary latent space used in the paper)
pca_discovery = decomposition.PCA(n_components=100)
pca_discovery.fit(img_array.T)
vox_ts_pca = pca_discovery.transform(img_array.T)
pca_discovery.explained_variance_ratio_.cumsum()

array([0.1750335 , 0.21433559, 0.23677755, 0.25723442, 0.27124828,
       0.28379784, 0.29423001, 0.30273892, 0.31103065, 0.31831005,
       0.32508828, 0.3312196 , 0.33704667, 0.34283736, 0.34802452,
       0.35290737, 0.35762634, 0.36213145, 0.36645256, 0.37040551,
       0.37412854, 0.37764351, 0.38108153, 0.38424912, 0.38714728,
       0.38994412, 0.39266673, 0.39530854, 0.39779825, 0.40021652,
       0.4025867 , 0.40488595, 0.40705126, 0.40918372, 0.41127842,
       0.41326095, 0.41520606, 0.41707585, 0.41889039, 0.4206175 ,
       0.42232146, 0.42393868, 0.42551882, 0.42705816, 0.42856839,
       0.43006309, 0.43151621, 0.43291489, 0.43429748, 0.4356772 ,
       0.43704164, 0.43835956, 0.43965986, 0.44093896, 0.44221421,
       0.44346178, 0.44468586, 0.4458818 , 0.44707348, 0.44824501,
       0.44940899, 0.45056437, 0.45171369, 0.45284175, 0.45396302,
       0.45504713, 0.45612134, 0.45718261, 0.45823468, 0.45927157,
       0.46029355, 0.46129735, 0.46229511, 0.46329142, 0.46427

In [ ]:
# pca on task voxel timeseries
pca = decomposition.PCA(n_components=100)
pca.fit(img_array_comb_disc)
vox_ts_pca = pca.transform(img_array_comb_disc)
pca.explained_variance_ratio_.cumsum()

In [158]:
# save or load PCA object
#joblib.dump(pca_discovery, 'pca_discovery_100comps.joblib') 
pca_discovery = joblib.load('pca_discovery_100comps.joblib') 
#joblib.dump(pca_validation, 'pca_validation_100comps.joblib') 
#pca_validation = joblib.load('pca_validation_100comps.joblib') 

['pca_validation_100comps.joblib']

In [ ]:
# project validation data into discovery latent space
vox_ts_pca_val_proj_disc = pca_discovery.transform(img_array.T)

In [30]:
# save PCA scores, aka gradient slope timeseries
#np.save('vox_ts_pca.npy', vox_ts_pca)
#np.save('vox_ts_pca_val.npy', vox_ts_pca)
#np.save('vox_ts_pca_val_proj_disc.npy', vox_ts_pca_val_proj_disc)
#np.save('vox_ts_pca_task.npy', vox_ts_pca)

In [156]:
# save PCA coefficients, aka region gradient weights
#np.save('vox_ts_pca_coeff.npy', pca.components_)
#np.save('vox_ts_pca_coeff_disc.npy', pca.components_)
#np.save('vox_ts_pca_coeff_val.npy', pca.components_)

In [31]:
# save gradient weight nii maps
for comp_num in range(100):
    vox_betas = np.zeros((91,109,91))
    vox_betas[gm_vox[0], gm_vox[1], gm_vox[2]] = pca.components_[comp_num,:]
    
    # create voxelwise beta feature nii file
    input = nib.load('MNI152_T1_2mm.nii')
    cur_affine = input.affine
    output = vox_betas*1000
    output_filename = 'pca_grad_maps/pca_grad_disc_%03d.nii' %(comp_num+1)
    #output_filename = 'pca_grad_maps/pca_grad_val_%03d.nii' %(comp_num+1)
    #output_filename = 'pca_grad_maps/pca_grad_disc_task_%03d.nii' %(comp_num+1)
    output_file = nib.Nifti1Image(output, cur_affine)
    nib.save(output_file, output_filename)

In [14]:
# pca on ROI timeseries for comparison to voxelwise PCA
pca = decomposition.PCA(n_components=20)
pca.fit(roi_img_means[:119500,:])
roi_ts_pca = pca.transform(roi_img_means[:119500,:])
pca.explained_variance_ratio_.cumsum()

array([0.35784946, 0.43564913, 0.49003337, 0.53338172, 0.56764932,
       0.59487169, 0.61555242, 0.63216762, 0.64722145, 0.65949975,
       0.67068073, 0.68071977, 0.68952695, 0.69764478, 0.705031  ,
       0.71197647, 0.71872957, 0.72493689, 0.73065036, 0.73577617])

In [32]:
# project all task data into discovery latent space
pca_discovery = joblib.load('pca_discovery_100comps.joblib')
vox_ts_pca_task_proj_disc = pca_discovery.transform(img_array_comb_disc)
#np.save('vox_ts_pca_task_proj_disc.npy', vox_ts_pca_task_proj_disc)

In [33]:
# validation task data
img_array_comb_val = np.vstack((img_array_wm[:,40000:].T,img_array_motor[:,27900:].T)).astype('float32')
img_array_comb_val = np.vstack((img_array_comb_val,img_array_language[:,31100:].T)).astype('float32')
img_array_comb_val = np.vstack((img_array_comb_val,img_array_emotion[:,17100:].T)).astype('float32')

In [34]:
# project validation task data into discovery latent space
vox_ts_pca_task_val_proj_disc = pca_discovery.transform(img_array_comb_val)
#np.save('vox_ts_pca_task_val_proj_disc.npy', vox_ts_pca_task_val_proj_disc)